##In this final notebook, therapist_interview.ipynb, we bring together all the components trained earlier to simulate a complete AI therapist session. The purpose of this notebook is to ask the user a series of reflective questions, detect the emotions behind their responses, generate supportive replies, and then summarize the user’s overall emotional state using a trained language model.

In [27]:
    from IPython.display import HTML, display

    def set_css():
      display(HTML('''
      <style>
        pre {
          white-space: pre-wrap;
        }
      </style>
      '''))
    get_ipython().events.register('pre_run_cell', set_css)

We begin by loading the two models required for this system. First, we unzip and load the DistilBERT emotion classifier, which was trained using the GoEmotions dataset. This model is responsible for detecting the emotional tone of each user response — emotions like sadness, joy, fear, or confusion. Next, we mount Google Drive and unzip the T5-based therapist response model, which was fine-tuned on the CounselChat dataset. This model is used to generate empathetic, therapist-style replies to user input.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -o /content/drive/MyDrive/t5_therapist_model.zip -d /content

Archive:  /content/drive/MyDrive/t5_therapist_model.zip
   creating: /content/t5_therapist_model/
   creating: /content/t5_therapist_model/checkpoint-981/
  inflating: /content/t5_therapist_model/checkpoint-981/optimizer.pt  
  inflating: /content/t5_therapist_model/checkpoint-981/config.json  
  inflating: /content/t5_therapist_model/checkpoint-981/trainer_state.json  
  inflating: /content/t5_therapist_model/checkpoint-981/generation_config.json  
  inflating: /content/t5_therapist_model/checkpoint-981/model.safetensors  
  inflating: /content/t5_therapist_model/checkpoint-981/scheduler.pt  
  inflating: /content/t5_therapist_model/checkpoint-981/rng_state.pth  
  inflating: /content/t5_therapist_model/checkpoint-981/training_args.bin  
   creating: /content/t5_therapist_model/checkpoint-500/
  inflating: /content/t5_therapist_model/checkpoint-500/optimizer.pt  
  inflating: /content/t5_therapist_model/checkpoint-500/config.json  
  inflating: /content/t5_therapist_model/checkpoint-5

In [ ]:
!unzip -o mental_health_model.zip

Archive:  mental_health_model.zip
  inflating: mental_health_model/vocab.txt  
  inflating: mental_health_model/tokenizer_config.json  
  inflating: mental_health_model/config.json  
  inflating: mental_health_model/special_tokens_map.json  
  inflating: mental_health_model/model.safetensors  
  inflating: mental_health_model/tokenizer.json  


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch
from collections import Counter

emotion_model = DistilBertForSequenceClassification.from_pretrained("./mental_health_model", local_files_only=True)
emotion_tokenizer = DistilBertTokenizerFast.from_pretrained("./mental_health_model", local_files_only=True)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

t5_tokenizer = T5Tokenizer.from_pretrained("./t5_therapist_model")
t5_model = T5ForConditionalGeneration.from_pretrained("./t5_therapist_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
emotion_model.to(device)
t5_model.to(device)

emotion_model.eval()
t5_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

Once both models are loaded and ready, we define the list of 28 emotion labels and set up a helper function to predict the emotion for a given input. Then we define the response generation function, which takes the user’s answer and feeds it to the T5 model to produce a supportive and relevant reply.

In [ ]:
label_names = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
    'remorse', 'sadness', 'surprise', 'neutral'
]
id2label = {i: name for i, name in enumerate(label_names)}

def predict_emotion(text):
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = emotion_model(**inputs).logits
        predicted_id = torch.argmax(logits, dim=1).item()
    return id2label[predicted_id]

def generate_therapist_reply(user_input):
    input_text = "question: " + user_input.strip()
    inputs = t5_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)
    output = t5_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    return t5_tokenizer.decode(output[0], skip_special_tokens=True).strip()


The main part of the notebook is the interview loop. Here, the AI asks the user five structured, therapist-style questions — such as “What made you feel unsettled recently?” or “What kind of support do you feel you're missing?” After each answer, the notebook predicts the emotion and generates a therapist-like response. All of this — the question, user answer, detected emotion, and AI reply — is logged into a session log.

In [28]:
questions = [
    "What made you feel uncomfortable or unsettled recently?",
    "When did you last feel truly understood?",
    "What thoughts do you keep coming back to lately?",
    "Is there something you wish people knew about how you’re feeling?",
    "What kind of support do you feel you're missing?"
]

session_log = []

for i, question in enumerate(questions, 1):
    print(f"\nQuestion {i}: {question}")
    answer = input("Your answer: ")
    emotion = predict_emotion(answer)
    therapist_reply = generate_therapist_reply(answer)

    print(f"Detected Emotion: {emotion}")
    print(f"Therapist Response: {therapist_reply}\n")

    session_log.append({
        "question": question,
        "answer": answer,
        "emotion": emotion,
        "response": therapist_reply
    })


Question 1: What made you feel uncomfortable or unsettled recently?
Your answer: I felt ignored in class.
Detected Emotion: annoyance
Therapist Response: I'm sorry to hear that you've been ignored by class. It's a good question to ask yourself if you are unsure of what is going on in your class and how you feel about it. If you haven't been able to do this, then you may want to know that there are some things you can do to help you.


Question 2: When did you last feel truly understood?
Your answer: Last week during a deep talk with my best friend.
Detected Emotion: neutral
Therapist Response: It sounds like you are having a deep conversation with your best friend. If you haven't had the time to talk to him or her, then he or she will be able to speak with you.


Question 3: What thoughts do you keep coming back to lately?
Your answer: That I'm not good enough.
Detected Emotion: disappointment
Therapist Response: I'm sorry to hear that you're not good enough. It sounds like you aren't

Finally, once the five questions are completed, we generate a final therapist insight. This is not rule-based or hardcoded — instead, we pass the sequence of detected emotions into the fine-tuned T5 model, which produces a reflective summary of what the user might be going through emotionally. The goal is to simulate how a therapist might recognize patterns and provide insight into someone’s emotional experience, without repeating the user’s exact words.

In [36]:
def generate_session_summary(session_log):
    condensed_inputs = ", ".join([entry["emotion"] for entry in session_log])

    prompt = (
        f"summary: The user expressed the following emotions during their therapy session: {condensed_inputs}. "
        "As a compassionate therapist, reflect on what these emotions may suggest about the user's inner experience, "
        "possible challenges, or emotional needs."
    )

    inputs = t5_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    output = t5_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=180,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    return t5_tokenizer.decode(output[0], skip_special_tokens=True).strip()

final_summary = generate_session_summary(session_log)
print("\nFinal Therapist Insight:")
print(final_summary)


Final Therapist Insight:
The user expressed a number of emotions during their therapy session, including annoyance, neutral, disappointment, contradiction. As compassionate therapists, reflect on what these emotions may suggest about the user's inner experience, possible challenges, or emotional needs.


In [39]:
def generate_session_summary(session_log):
    emotions = [entry["emotion"] for entry in session_log]
    emotion_summary = ", ".join(emotions)

    prompt = (
        f"summary: The user displayed these emotions in their therapy session: {emotion_summary}. "
        "As a therapist, reflect on what emotional pattern or root struggle this might represent."
    )


    inputs = t5_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    output = t5_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=160,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    return t5_tokenizer.decode(output[0], skip_special_tokens=True).strip()

final_summary = generate_session_summary(session_log)
print("\nFinal Therapist Insight:")
print(final_summary)



Final Therapist Insight:
The user displayed these emotions in a therapy session: annoyance, neutral, disappointment, contradiction.Also, if you are able to reflect on what emotional pattern or root struggle this might represent, you may want to look at how you feel about your emotions during your session.
